In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import datetime
from gee_scripts.get_sources import get_explanatory_composite
from gee_scripts.parameters import explain_vars
from gee_scripts.get_sources import get_s1_dates
from gee_scripts.gee import export_classifier
from gee_scripts.directories import get_export_folder
from gee_scripts import init_ee

import ee


Earth Engine initialized successfully, with ee-indonesia-gwl


## 1. Train the classifier in GEE and save the model as asset

In [3]:
df = pd.read_csv("data/9_clean_training_data/all_training_data_with_extra_and_locations_and_precipSum.csv", parse_dates=["date"])

In [4]:
# This table contains the training data for the model
training_data = ee.FeatureCollection("projects/ee-indonesia-gwl/assets/all_training_data_with_extra_and_locations_and_precipSum")

In [5]:
best_kalimantan_phus = [297, 351, 352, 256, 340]
training_data = training_data.filter(ee.Filter.inList("phu_id", best_kalimantan_phus))
training_data.size().getInfo()

3837

In [19]:
# model_gee_id = export_classifier(training_data, "kalimantan_phu_model_297_351_352_256_340")
model_gee_id = "projects/ee-indonesia-gwl/assets/gwl-modeling/models/RandomForest_kalimantan_phu_model_297_351_352_256_340_trees_250"

## 2. Estimate using the trained model

https://code.earthengine.google.com/1fd31fe53d5a8cf8b812552f901325a0

## 2.1. Set parameters

In [8]:
target_phu_number = 351
target_phu = ee.FeatureCollection("projects/ee-indonesia-gwl/assets/all_phus_numbered").filter(
    ee.Filter.eq("phu_id", target_phu_number)).first(\
).geometry().bounds()

In [9]:
# This lie will take some time to run, because it will do a GEE request
all_s1_dates = get_s1_dates(aoi=target_phu)

In [10]:
all_str_date = [
    datetime.datetime.fromtimestamp(date/1000).strftime('%Y-%m-%d') for date in all_s1_dates
]

In [11]:
dates_df = pd.DataFrame(all_str_date, columns=["date"])
# drop duplicates
dates_df = dates_df.drop_duplicates()
dates_df["date"] = pd.to_datetime(dates_df["date"])
# Only keep the dates after 2019
dates = dates_df[dates_df["date"] > "2019-09-22"]
len(dates)

all_str_date = dates["date"].dt.strftime('%Y-%m-%d').tolist()
len(all_str_date)

264

In [20]:
def export_estimation(model_name, aoi_name, ee_aoi, str_date, ee_classifier, ):
    """Export the estimated GWL image for a given model and target date."""

    # Get explanatory composite closest to target date
    image = get_explanatory_composite(
        target_date=str_date, 
        ee_region=ee_aoi, 
    ).select(explain_vars)

    output_image_name = f"{aoi_name}_{str_date}"
    estimated_image = image.select(explain_vars).classify(ee_classifier).set({"model": model_name, "date": str_date})

    # create export task
    task = ee.batch.Export.image.toAsset(
        **{
            "image": estimated_image,
            "description": output_image_name,
            "assetId": str(get_export_folder(output_folder=f"estimation/best_model")/output_image_name),
            "region": ee_aoi,
            "scale": 100,
        }
    ).start()

In [21]:
aoi_name = f"PHU_{target_phu_number}"
ee_classifier = ee.Classifier.load(model_gee_id)
model_name = "kalimantan_phu_model_297_351_352_256_340"
phus = ee.FeatureCollection("projects/ee-indonesia-gwl/assets/All_phus_numbered")

tasks = [
    export_estimation(model_name, aoi_name, target_phu, target_date, ee_classifier) 
    for target_date 
    in all_str_date
]

target_date 2019-09-28
start_date 2019-09-27 00:00:00
end_date 2019-09-29 00:00:00
target_date 2019-10-10
start_date 2019-10-09 00:00:00
end_date 2019-10-11 00:00:00
target_date 2019-10-16
start_date 2019-10-15 00:00:00
end_date 2019-10-17 00:00:00
target_date 2019-10-22
start_date 2019-10-21 00:00:00
end_date 2019-10-23 00:00:00
target_date 2019-10-27
start_date 2019-10-26 00:00:00
end_date 2019-10-28 00:00:00
target_date 2019-11-03
start_date 2019-11-02 00:00:00
end_date 2019-11-04 00:00:00
target_date 2019-11-08
start_date 2019-11-07 00:00:00
end_date 2019-11-09 00:00:00
target_date 2019-11-15
start_date 2019-11-14 00:00:00
end_date 2019-11-16 00:00:00
target_date 2019-11-20
start_date 2019-11-19 00:00:00
end_date 2019-11-21 00:00:00


In [ ]:
[task.start() for task in tasks]

In [ ]:
list_of_images = ee.data.listImages("projects/ee-indonesia-gwl/assets/gwl-modeling/estimation/kalimantan_phu_model_297_351_352_256_340")
list_of_images = [image["id"] for image in list_of_images["images"]]

In [ ]:
from pathlib import Path

image_collection = "projects/ee-indonesia-gwl/assets/gwl-modeling/estimation/best_model"
destination_id = Path(image_collection)/Path(list_of_images[0]).name
str(destination_id)

In [ ]:
ic = ee.ImageCollection("projects/ee-indonesia-gwl/assets/gwl-modeling/estimation/best_model")

In [ ]:
ic.first().get("date").getInfo()

In [ ]:
running = [operation for operation in ee.data.listOperations() if operation["metadata"]["state"] == "RUNNING"]
pending = [operation for operation in ee.data.listOperations() if operation["metadata"]["state"] == "PENDING"]

print(f"Running: {len(running)}, Pending: {len(pending)}")

In [ ]:
running

In [ ]:
[ee.data.cancelOperation(operation["name"]) for operation in pending]

In [ ]:
image_ids = ic.toList(ic.size()).getInfo()
image_ids = [image["id"] for image in image_ids]

for image_id in image_ids:

    image = ee.Image(image_id)
    date = image.get("date")
    image.set({"system:time_start": date})

    ee.data.updateAsset(image_id, image, [])

    break    